In [2]:
import os
import re
import pandas as pd

## Unificar archivos
Toma todos los achivos c*.txt de la carpeta alumnos y lo guarda normalizados en alumnos.md

Usa el nombre del archivo para averiguar el numero de comisión 

In [28]:

def unificar_archivos():
    # Directorio de los archivos de alumnos
    directory = 'alumnos'

    # Crear o abrir el archivo de salida
    with open('alumnos.md', 'w') as output:
        # Obtener la lista de archivos en el directorio
        files = os.listdir(directory)
        
        # Filtrar solo los archivos que comienzan con 'c' y terminan con '.txt'
        files = [f for f in files if f.startswith('c') and f.endswith('.txt')]
        
        output.write("# Alumnos LAB4.2024 \n\n")
        for file in files:
            # Obtener el número de comisión del nombre del archivo
            commission_number = re.search(r'\d+', file).group()
            
            # Escribir el encabezado de la comisión
            output.write(f"## Comision {commission_number}\n")
            
            # Leer el archivo línea por línea
            with open(os.path.join(directory, file), 'r') as input_file:
                lines = input_file.readlines()
                
                for index, line in enumerate(lines, start=1):
                    # Separar el puntaje del nombre completo
                    parts = line.strip().split(' ', 1)
                    if len(parts) < 2:
                        continue
                    score, full_name = parts
                    
                    # Eliminar el punto del puntaje
                    score = score.replace('.', '')
                    
                    # Escribir la línea formateada en el archivo de salida
                    output.write(f"{index:2}. {score} {full_name}\n")
            
            # Añadir una línea en blanco para separar las comisiones
            output.write("\n")

## Cargar alumnos

Toma los alumnos que estan en alumnos.md y lo carga en un datatable.
El formato del archivo debe ser
```
## Comision N
1. Legajo Apellido, Nombre
```

In [29]:

def cargar_alumos(origen='alumnos.md'):
    # Leer el archivo
    with open(origen, 'r') as file:
        lines = file.readlines()
    
    # Inicializar listas para almacenar los datos
    comisiones = []
    numeros_orden = []
    legajos = []
    apellidos = []
    nombres = []
    
    # Variables para almacenar la comisión actual
    comision_actual = None
    
    # Procesar cada línea del archivo
    for line in lines:
        line = line.strip()
        if line.startswith('## Comision'):
            # Extraer el número de la comisión
            comision_actual = int(line.split(' ')[-1])
        elif line and line[0].isdigit():
            # Extraer el número de orden, legajo, apellido y nombre
            match = re.match(r'(\d+)\.\s+(\d+)\s+([^,]+),\s+(.+)', line)
            if match:
                numero_orden = int(match.group(1))
                legajo = match.group(2)
                apellido = match.group(3)
                nombre = match.group(4)
                
                # Almacenar los datos en las listas
                comisiones.append(comision_actual)
                numeros_orden.append(numero_orden)
                legajos.append(legajo)
                apellidos.append(apellido)
                nombres.append(nombre)
    
    # Crear el DataFrame
    data = {
        'comision': comisiones,
        'orden': numeros_orden,
        'legajo': legajos,
        'apellido': apellidos,
        'nombre': nombres
    }
    df = pd.DataFrame(data)
    
    return df

# Uso de la función


## Detectar datos duplicados

Revisa si los legajos estan duplicados (error), y como curiosidad si hay nombre y apellidos repetidos.

In [59]:
def detectar_duplicados(d):
    # Listar registros con legajo duplicado
    legajos_duplicados = d[d['legajo'].duplicated()]
    print("Registros con legajo duplicado:")
    if len(legajos_duplicados) == 0:
        print("No hay legajos duplicados")
    else:
        print("ERROR: Se encontraron legajos duplicados")
        print(legajos_duplicados)

    # Listar registros con nombre duplicado
    nombres_duplicados = d[d['nombre'].duplicated(keep=False)]
    nombres_duplicados.sort_values(by='nombre', inplace=True)
    num_nombres = len(nombres_duplicados['nombre'].unique())

    print(f"\nHay {num_nombres} Nombres duplicados en {len(nombres_duplicados)} registros")
    print("Registros con nombre duplicado:")
    print(nombres_duplicados)

    # Listar registros con apellido duplicado
    apellidos_duplicados = d[d['apellido'].duplicated(keep=False)]
    apellidos_duplicados.sort_values(by='apellido', inplace=True)

    num_apellidos = len(apellidos_duplicados['apellido'].unique())

    print(f"\nHay {num_apellidos} Apellidos duplicados en {len(apellidos_duplicados)} registros")
    print("Registros con apellido duplicado: ")
    print(apellidos_duplicados)



In [61]:
unificar_archivos()
a = cargar_alumos()
detectar_duplicados(a)

Registros con legajo duplicado:
No hay legajos duplicados

Hay 6 Nombres duplicados en 12 registros
Registros con nombre duplicado:
     comision  orden legajo         apellido         nombre
16          9     17  59074          Teseyra   Juan Ignacio
77          7     25  58934      Quiroga Gil   Juan Ignacio
5           9      6  59313         Pisentte     Juan Pablo
131         5     48  59064           Urbani     Juan Pablo
69          7     17  59044     Naranjo Sosa          Lucas
109         5     26  58727          Lazarte          Lucas
10          9     11  59933          Saucedo   Lucas Nahuel
19          3      1  58931          Cabrera   Lucas Nahuel
40          3     22  58763          Donelli        Luciano
58          7      6  59158           Corral        Luciano
67          7     15  59134           Molina  Tomás Agustín
91          5      8  58895  Flores Gonzalez  Tomás Agustín

Hay 14 Apellidos duplicados en 30 registros
Registros con apellido duplicado: 
     com

/var/folders/6n/z412hmln0wn4g5vby875r5s00000gn/T/ipykernel_25092/789148701.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nombres_duplicados.sort_values(by='nombre', inplace=True)
/var/folders/6n/z412hmln0wn4g5vby875r5s00000gn/T/ipykernel_25092/789148701.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apellidos_duplicados.sort_values(by='apellido', inplace=True)
